# Part 4 - Query Parcels

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-4---Query-Parcels" data-toc-modified-id="Part-4---Query-Parcels-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 4 - Query Parcels</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Parcel-Feature-Relationships-and-Querying" data-toc-modified-id="Parcel-Feature-Relationships-and-Querying-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Parcel Feature Relationships and Querying</a></span><ul class="toc-item"><li><span><a href="#Records" data-toc-modified-id="Records-1.0.1.1"><span class="toc-item-num">1.0.1.1&nbsp;&nbsp;</span>Records</a></span></li><li><span><a href="#Parcel-features" data-toc-modified-id="Parcel-features-1.0.1.2"><span class="toc-item-num">1.0.1.2&nbsp;&nbsp;</span>Parcel features</a></span></li><li><span><a href="#Find-a-parcel-record-by-its-name" data-toc-modified-id="Find-a-parcel-record-by-its-name-1.0.1.3"><span class="toc-item-num">1.0.1.3&nbsp;&nbsp;</span>Find a parcel record by its name</a></span></li></ul></li><li><span><a href="#Find-parcel-features-by-Record-GlobalID" data-toc-modified-id="Find-parcel-features-by-Record-GlobalID-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Find parcel features by Record GlobalID</a></span></li><li><span><a href="#API-Ref-Documentation" data-toc-modified-id="API-Ref-Documentation-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>API Ref Documentation</a></span></li></ul></li></ul></li></ul></div>

### Parcel Feature Relationships and Querying

This notebook demonstrates the following:

1. How to query a feature layer
2. How to find parcel features within a parcel record.
3. How to find parcel features by an attribute.
4. Build up a dict of parcel features for Parcel Fabric capabilities.

#### Records
A Parcel Fabric contains a Records which represents the legal document information about a parcel or set of parcels.  The geometry of a record is the outer boundary of any parcel polygon features associated with a particular record.  The record feature and parcel feature(s) are related by GUID fields (GlobalID) in each.

|Field Name  |Type  |
|------------|------|
|GlobalID    |GUID  |

#### Parcel features
A parcel type consists of a pair of feature classes.  
- A polygon feature class representing the area of a parcel.
- A COGO enabled line feature class representing the boundary of a parcel.

Parcel features (both lines and polygons) relate to a Record and to themselves:

|Field Name      |Type       |
|----------------|-----------|
|CreatedByRecord |GUID       |
|RetiredByRecord |GUID       |

In [1]:
from arcgis.gis import GIS
from arcgis.features.layer import FeatureLayerCollection
from arcgis.features.layer import FeatureLayer

In [2]:
base_server_url = (
    "https://rextapilnx02eb.mysite.com/server/rest/services/WashingtonCounty/"
)
gis = GIS(
    "https://myenterprisesite.com/portal/",
    "my_user_name",
    "my_secure_password",
    verify_cert=False,
)

# generate the enpoint urls for feature server, version management and parcel fabric
service_endpoints = ["FeatureServer", "VersionManagementServer"]
service_urls = {url: base_server_url + url for url in service_endpoints}

parcel_fabric_flc = FeatureLayerCollection(service_urls["FeatureServer"], gis)
parcel_fabric_flc.layers[0].properties.name

Setting `verify_cert` to False is a security risk, use at your own risk.


'WashCoFabricSubset'

In [3]:
# Get the URL of the Records feature layer
records_fl_url = [n for n in parcel_fabric_flc.layers if n.properties.name == "Records"]
records_fl_url = records_fl_url[0].url
records_fl_url

'https://rextapilnx02eb.mysite.com/server/rest/services/WashingtonCounty/FeatureServer/1'

#### Find a parcel record by its name

- Access the Records feature layer
- Create a WHERE clause and list of fields to return
- Query the records feature layer
- Return the result as a spatial data frame
 

In [4]:
record_name = "Record001"  # name to search
out_fields = ["name", "globalId"]  # fields to return

where = f"name = '{record_name}'"

records_fl = FeatureLayer(records_fl_url, gis)
record_row = records_fl.query(
    where=where,
    out_fields=out_fields,
).sdf

record_row

,objectid,name,globalid,SHAPE
0,123,Record001,{4B753DC5-AEDD-4703-AE56-B236EBB5DEFB},"{""rings"": [[[2474616.7778283805, 503108.655806..."


### Find parcel features by Record GlobalID
In the cell above, the Records feature layer was queried by using its name.  The GlobalID value for 'Record001' is now available to use in a search for its associated parcel features.

The proceeding cells will show the following:
1. Extract the globalID of the record
2. Access the parcel polygon feature layer
3. Query the features in the parcels feature layer associated to the record

`parcels(CreatedByRecord) = records(GlobalID)`

4. Draw the subset on a map

In [5]:
record_globalid = record_row.at[0, "globalid"]
record_globalid

'{4B753DC5-AEDD-4703-AE56-B236EBB5DEFB}'

In [6]:
# Get the URL of the parcel polygon feature layer
parcels_fl_url = [n for n in parcel_fabric_flc.layers if n.properties.name == "Tax_PF"]
parcels_fl_url = parcels_fl_url[0].url
parcels_fl_url

'https://rextapilnx02eb.mysite.com/server/rest/services/WashingtonCounty/FeatureServer/15'

In [7]:
out_fields = ["name", "globalId"]
where_clause = f"CreatedByRecord = '{record_globalid}'"
parcels_fl = FeatureLayer(parcels_fl_url, gis)

parcels_subset = parcels_fl.query(where=where_clause, out_fields=out_fields)
parcels_subset

<FeatureSet> 12 features

### API Ref Documentation

- [ArcGIS Python API - Version Manager](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#versionmanager)
- [ArcGIS Python API - Parcel Fabric Manager](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#parcelfabricmanager)
- [ArcGIS REST API - VersionManagementServer](https://developers.arcgis.com/rest/services-reference/enterprise/version-management-service.htm)
- [ArcGIS REST API - ParcelFabricServer](https://developers.arcgis.com/rest/services-reference/enterprise/overview-of-parcel-fabric-sevices.htm)
- [ArcGIS Pro - Branch Versioning Scenarios](https://pro.arcgis.com/en/pro-app/latest/help/data/geodatabases/overview/branch-version-scenarios.htm)